# 0. 引用

In [1]:
%load_ext autoreload
%autoreload 2
import analyze_review_reply
import analyze_review
import pandas as pd
from glob import glob
# TODO 合并analyze_review_reply和analyze_review.py

# 1. 分析实验中采样的数据

## 1.1 UI review的采样数据及统计

In [2]:
all_label = pd.read_csv("label_app_review.csv", encoding="utf_8_sig")
all_label["SUBCATEGORY"] = all_label["SUBCATEGORY"].fillna("FALSE")

analyze_review.count_category(all_label)
analyze_review.count_subcategory(all_label)

统计各个UI category （包括多标签）
{&#39;appearance&#39;: 329, &#39;interaction&#39;: 303, &#39;experience&#39;: 369, &#39;others&#39;: 468, &#39;FALSE&#39;: 22}
统计各个UI types（包括多标签）
{&#39;layout&#39;: 62, &#39;color&#39;: 50, &#39;typography&#39;: 73, &#39;iconography&#39;: 73, &#39;image&#39;: 73, &#39;navigation&#39;: 75, &#39;notification&#39;: 117, &#39;motion&#39;: 35, &#39;gesture&#39;: 70, &#39;accessibility&#39;: 22, &#39;redundancy&#39;: 112, &#39;customization limitation&#39;: 108, &#39;advertisement&#39;: 53, &#39;feedback&#39;: 94, &#39;generic evaluation&#39;: 232, &#39;comparative review&#39;: 142, &#39;material&#39;: 95, &#39;FALSE&#39;: 22}
False positive的数量以及比例
22
0.014976174268209666


## 1.2 分析UI review reply的采样数据及统计

In [3]:
files = sorted(glob("review_reply_*.xlsx"))
data_list = [pd.read_excel(f) for f in files]
all_review_reply = pd.concat(data_list)
all_review_reply.dropna(subset=["REPLY", "CATEGORY"], inplace=True)

analyze_review_reply.count_category(all_review_reply)
analyze_review_reply.count_reply(all_review_reply)

统计各个UI category （包括多标签）
{&#39;appearance&#39;: 0, &#39;interaction&#39;: 0, &#39;experience&#39;: 0, &#39;others&#39;: 0, &#39;False&#39;: 0}
统计各个pattern（包括多标签）
{&#39;apology or appreciation&#39;: 255, &#39;advice&#39;: 106, &#39;information request&#39;: 141, &#39;justify&#39;: 70, &#39;promise&#39;: 68, &#39;inform&#39;: 59, &#39;unspecify&#39;: 34, &#39;False&#39;: 12}
False positive的数量以及比例
12
0.015463917525773196


In [4]:
print(all_label.columns)
print(all_label["PRIMARY_CATEGORY"].value_counts())
print(all_label[all_label["PRIMARY_CATEGORY"] == "others "]["PRIMARY_CATEGORY"])


Index([&#39;REVIEW_ID&#39;, &#39;APP_ID&#39;, &#39;PACKAGE_NAME&#39;, &#39;RATING&#39;, &#39;REVIEW_DATE&#39;,
       &#39;REVIEW_TITLE&#39;, &#39;REVIEW_TEXT&#39;, &#39;REPLY_TEXT&#39;, &#39;REPLY_DATE&#39;,
       &#39;REVIEWER_NAME&#39;, &#39;CATEGORY&#39;, &#39;PRIMARY_CATEGORY&#39;, &#39;SUBCATEGORY&#39;,
       &#39;PRIMARY_SUBCATEGORY&#39;, &#39;REPLY&#39;, &#39;HANDLED&#39;, &#39;IS_FREE&#39;],
      dtype=&#39;object&#39;)
others         468
experience     362
appearance     318
interaction    297
FALSE           22
Name: PRIMARY_CATEGORY, dtype: int64
Series([], Name: PRIMARY_CATEGORY, dtype: object)


In [5]:
def f(x):
    name = x["PRIMARY_SUBCATEGORY"].value_counts().index[0]
    # print(name)
    # print(len(x))
    low_rating = len(x[x["RATING"] < 3])
    ratio = low_rating / len(x)
    # message = "Category: %s, low rating ratio: %.2f" % (name, ratio)
    # print(message)
    return ratio

free = all_label[all_label["IS_FREE"] == 1]
print(len(free))
non_free = all_label[all_label["IS_FREE"] == 0]
print(len(non_free))
print("All_label")
all_label.groupby(["PRIMARY_SUBCATEGORY"]).apply(f)

798
669
All_label


PRIMARY_SUBCATEGORY
accessibility               0.380952
advertisement               0.653846
color                       0.152174
comparative review          0.901408
customization limitation    0.215686
feedback                    0.526882
generic evaluation          0.155172
gesture                     0.540984
iconography                 0.342857
image                       0.382353
layout                      0.440678
material                    0.294737
motion                      0.323529
navigation                  0.328767
notification                0.389381
redundancy                  0.428571
typography                  0.319444
dtype: float64

In [6]:
print("FREE")
free_ratio = free.groupby(["PRIMARY_SUBCATEGORY"]).apply(f)
print("NON_FREE")
non_free_ratio = non_free.groupby(["PRIMARY_SUBCATEGORY"]).apply(f)

FREE
NON_FREE


In [7]:
from scipy.stats import spearmanr
data = pd.DataFrame({"free_ratio": free_ratio, "non_free_ratio": non_free_ratio})
print(data.head())
print(data.corr())
spearmanr(free_ratio, non_free_ratio)

                          free_ratio  non_free_ratio
PRIMARY_SUBCATEGORY                                 
accessibility               0.363636        0.400000
advertisement               0.659574        0.600000
color                       0.166667        0.136364
comparative review          0.924812        0.555556
customization limitation    0.241935        0.175000
                free_ratio  non_free_ratio
free_ratio        1.000000        0.545204
non_free_ratio    0.545204        1.000000


SpearmanrResult(correlation=0.5122549019607844, pvalue=0.03552652411689097)

In [22]:
reviews = pd.read_csv("label_app_review.csv")
print(reviews.columns)
high_rank = reviews[reviews["RATING"] > 3][['RATING', 'REVIEW_ID', 'REVIEW_TEXT']]
for index, r in high_rank[10:50].iterrows():
    print(r["RATING"])
    print(r["REVIEW_ID"])
    print(r["REVIEW_TEXT"])

Index([&#39;REVIEW_ID&#39;, &#39;APP_ID&#39;, &#39;PACKAGE_NAME&#39;, &#39;RATING&#39;, &#39;REVIEW_DATE&#39;,
       &#39;REVIEW_TITLE&#39;, &#39;REVIEW_TEXT&#39;, &#39;REPLY_TEXT&#39;, &#39;REPLY_DATE&#39;,
       &#39;REVIEWER_NAME&#39;, &#39;CATEGORY&#39;, &#39;PRIMARY_CATEGORY&#39;, &#39;SUBCATEGORY&#39;,
       &#39;PRIMARY_SUBCATEGORY&#39;, &#39;REPLY&#39;, &#39;HANDLED&#39;, &#39;IS_FREE&#39;],
      dtype=&#39;object&#39;)
5
113343640
It speaks for itself easy to use and just like the reall deal. Thought it is free. If we dont have coins then we must purchase. Otherwise must wait upto 4 hours for free bonuses
5
110338463
Excellent and easy to use
5
96838445
I&#39;m currently in school for basic electrical training and this game has helped me understand logic gates more than my teacher ever could! Thank you a thousand times over!! P. S. You should have a screen pop up explaining the spinning yinyang as an inverter.
5
102540956
Very easy to use and well made.
5
109867069
Great a

### 在论文中使用的一个例子
Rank: 5

Reveiw ID：110498383

Text：I'm giving it 5/5 again because recent changes restored old filter icons (opposed to previews which were terrible to work with). Additionally fine tuning option has been finally introduced to let us manually set the parameters for low-colour filters. It seems to be stable aswell so I can finally recommend it. Good job guys.